In [1]:
import pct
from torch.utils.data import Dataset, DataLoader

In [2]:
train_ds = pct.DummyPointCloudDataset(n_samples=800, n_points=512)
val_ds = pct.DummyPointCloudDataset(n_samples=200, n_points=512)

train_loader = DataLoader(train_ds, batch_size=pct.TrainConfig.batch_size, shuffle=True, num_workers=0, collate_fn=pct.collate_batch)
val_loader = DataLoader(val_ds, batch_size=pct.TrainConfig.batch_size, shuffle=False, num_workers=0, collate_fn=pct.collate_batch)

In [3]:
model = pct.PointCloudTransformer(
        in_channels=3,
        d_model=pct.TrainConfig.d_model,
        depth=pct.TrainConfig.depth,
        n_heads=pct.TrainConfig.n_heads,
        k=pct.TrainConfig.k,
        latent_dim=pct.TrainConfig.latent_dim,
        out_dim=pct.TrainConfig.out_dim,
        drop=pct.TrainConfig.drop,
    )

In [4]:
pct.train_regression(model, train_loader, val_loader, epochs=pct.TrainConfig.epochs, lr=pct.TrainConfig.lr)

/Users/joshuahellewell/miniforge3/envs/pct/lib/python3.13/site-packages/pct/trainer.py:37: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device.startswith('cuda')))
/Users/joshuahellewell/miniforge3/envs/pct/lib/python3.13/site-packages/pct/trainer.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.startswith('cuda'))):
/Users/joshuahellewell/miniforge3/envs/pct/lib/python3.13/site-packages/pct/trainer.py:63: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.startswith('cuda'))):


Epoch 001 | train 0.0314 | val 0.0004


KeyboardInterrupt: 

In [6]:
points, targets = train_ds[2]

pct.visualize_point_cloud(points, title=f"Noisy Sphere (r={targets.item():.2f})")